In [ ]:
!pip install transformers -q
!pip install streamlit -q
!pip install openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import openai, requests, os
from anthropic import Anthropic

openai.api_key = ''
client = anthropic.Anthropic(
    api_key=''
)
MODEL_NAME = "claude-3-opus-20240229"

# Intro Text
st.markdown('# PDF summarization using Generative AI')
st.header('''PDF size must be below 200MB. Only 1 PDF per time, time will vary depending on size''')

input_file = st.file_uploader('Upload a PDF file')

# Save uploaded file
if input_file:
  with open(os.path.join(input_file.name),'wb') as f:
        f.write(input_file.getbuffer())
  st.success('Saved File')

st.markdown('# Summary')

def summarise_file(prompt,model = 'gpt-3.5-turbo'):
  st.success('Summary loading, please wait...')
  # Connect to API endpoint
  URL = 'https://api.openai.com/v1/chat/completions'

  parameters = {
      'model':model,
      'messages': [{'role':'user', 'content':prompt}],
      'temperature': 1,
      'top_p': 1,
      'frequency_penalty':1,
      'presence_penalty':1
  }

  headers = {
      'Content-Type': 'application/json',
      'Authorization': f'Bearer {openai.api_key}'
  }

  response = requests.post(URL, headers=headers, json=parameters, stream=False)
  json_response = response.json()
  summary = json_response['choices'][0]['message']['content'].strip()
  return summary


# def keywords(prompt,model = 'gpt-3.5-turbo'):
#   # Connect to API endpoint
#   URL = 'https://api.openai.com/v1/chat/completions'

#   parameters = {
#       'model':model,
#       'messages': [{'role':'user', 'content':prompt}],
#       'temperature': 1,
#       'top_p': 1,
#       'frequency_penalty':1,
#       'presence_penalty':1
#   }

#   headers = {
#       'Content-Type': 'application/json',
#       'Authorization': f'Bearer {openai.api_key}'
#   }

#   response = requests.post(URL, headers=headers, json=parameters, stream=False)
#   json_response = response.json()
#   summary = json_response['choices'][0]['message']['content'].strip()
#   return summary

def get_completion(client, prompt):
    return client.messages.create(
        model=MODEL_NAME,
        max_tokens=2048,
        messages=[{
            "role": 'user', "content":  prompt
        }]
    ).content[0].text

if input_file:
  prompt = f'''
  ### TASK
  You are an expert in text analysis and creating abstracts from research papers. Your task is to analyse a research paper I'm going
  to give you and create an abstract for it.
  ### INPUTS
  Research Paper: {input_file}
  ### OUTPUT
  The abstract must answer the following questions: What is the paper about? Why is the paper important? What methods were used?
  What industry partners were involved? What are the paper's findings? Why are the papers findings important?
  '''
  text_area = summarise_file(prompt)

  # Show Summary
  st.success('Summary ready!')
  st.text_area(label ="",value= text_area, placeholder="Please upload a PDF to get it's summary", height = 400)
  st.markdown('')


  # st.success('Recommendations loading, please wait...')

  # prompt = f'''
  # ### TASK
  # You are an expert in text analysis and keyword extraction. Your task is to analyse a research paper I'm going
  # to give you and extract from it the top scientific keywords that are most representative of its content. Then you're going to convert the
  # keywords to lower case and search the provided URL and return the top 5 results. The url you will use is
  # http://export.arxiv.org/api/query?search_query=all:keyword&start=0&max_results=5. If the keyword is more than one word seperate with a
  # + symbol such as machine+learning.
  # ### INPUTS
  # Summary: {input_file}
  # ### OUTPUT
  # The list of results you're going to generate should be in descending order from the most relevant to the least relevant.
  # '''
  # test = keywords(prompt)
  # # Show Recommendations
  # st.success('Recommendations ready!')
  # st.text_area(label ="",value= test, placeholder="Please upload a PDF to get it's summary", height = 400)
  # st.markdown('')

  completion = get_completion(client,
      f"""Here is an academic paper: <paper>{input_file}</paper>

  Please do the following:
  1. Summarise the research paper in such a way that the reader won't need to read the whole paper.
  """
  )
  print(completion)
  st.text_area(label ="",value= completion, placeholder="Please upload a PDF to get it's summary", height = 400)
  st.markdown('')

else:
  st.text_area(label ="",value= "Please upload a PDF to get it's summary", placeholder="Please upload a PDF to get it's summary", height = 400)

Overwriting app.py


You are an expert in creating abstracts from research papers and finding similar abstracts. Your task is to analyse a research paper I'm going
  to give you and create an abstract that is most representative of its content especially the points made in the first paragraph. You will th
  The url you will use is http://export.arxiv.org/api/query?search_query=all:keyword&start=0&max_results=5. If the keyword is more
  than one word seperate with a + symbol such as machine+learning.

### TASK
  You are an expert in text analysis and summarisation with research papers. Your task is to analyse a research paper I'm going to give you
  and provide a detailed summary that is most representative of its content. You must use the Key Concept Clarity method to read the paper and
  highlight all of the key talking points of the paper.
  ### INPUTS
  Research Paper: {input_file}
  ### OUTPUT
  The output should be structured so as to identify and concisely explain the main ideas in the document. Each section should consist of the
  name of the talking point, a detailed definition of the talking point and a short summary of the talking point.
  Aim to provide at least four talking points. Here is how each talking point should be structured:
  Name of the talking point
  Explanation: Define in detail what is the talking point
  Overview: Provide a short summary of the talking point including its challenges ,if any, and findings.
  '''

Rerun this if error

In [ ]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 1.045s


In [ ]:
!curl ipv4.icanhazip.com

35.247.29.50


In [ ]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501

npx: installed 22 in 2.603s
your url is: https://witty-spoons-say.loca.lt
^C


In [1]:
!pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.4/865.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [8]:
from anthropic import Anthropic
client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=''
)
MODEL_NAME = "claude-3-opus-20240229"

In [9]:
def get_completion(client, prompt):
    return client.messages.create(
        model=MODEL_NAME,
        max_tokens=2048,
        messages=[{
            "role": 'user', "content":  prompt
        }]
    ).content[0].text

In [10]:
text = '/content/Operational_range_bounding_of_spectroscopy_models_with_anomaly_detection_draft.pdf'

In [11]:
completion = get_completion(client,
    f"""Here is an academic paper: <paper>{text}</paper>

Please do the following:
1. Summarise the research paper in such a way that the reader won't need to read the whole paper.
"""
)
print(completion)

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}